# Pré-Requisitos

## PROV - https://pypi.org/project/prov/

In [ ]:
!pip install prov

## Sweetviz - https://pypi.org/project/sweetviz/

In [ ]:
!pip install sweetviz

# Bibliotecas

In [ ]:
import sweetviz as sv
import pandas as pd
import prov
import bs4

# Reprodutibilidade

## Mostra as versões das bibliotecas usadas

In [ ]:
import sys

print('Versão do Pandas:', pd.__version__)
print('Versão do Beautiful Soup:', bs4.__version__)
print('Versão do PROV:', prov.__version__)
print('Versão do Sweetviz:', sv.__version__)
print('Versão do Python:', sys.version)

## Sistema Operacional

In [ ]:
!cat /etc/os-release

### Versão do kernel

In [ ]:
!cat /proc/version

### Arquitetura (32 ou 64 bits)

In [ ]:
!lscpu

# Provenance

In [ ]:
# https://nbviewer.jupyter.org/github/trungdong/notebooks/blob/master/PROV%20Tutorial.ipynb
import prov.model as model
# Create a new provenance document
doc = model.ProvDocument()
# Declaring namespaces
#doc.add_namespace('prov', 'http://www.w3.org/ns/prov#')
doc.add_namespace('kagg', 'https://www.kaggle.com/')
# Criando entidades
dataset = doc.entity('kagg:fake-news', {model.PROV_TYPE : 'kagg:dataset', model.PROV_LOCATION : 'https://www.kaggle.com/mrisdal/fake-news',
                                        'kagg:title' : 'Getting Real about Fake News', 'kagg:resume' : 'Text & metadata from fake & biased news sources around the web',
                                        'kagg:usability' : '7.1', 'kagg:license' : 'CC0: Public Domain', 'kagg:tags' : 'arts and entertainment, news, politics, languages',
                                        'kagg:records' : '12.999', 'kagg:columns' : '20'})
zip = doc.entity('kagg:FakeNewData.zip', {model.PROV_TYPE : 'File', 'kagg:size' : '19.4 MB'})
csv = doc.entity('kagg:fake.csv', {model.PROV_TYPE : 'File', 'kagg:size' : '54.05 MB'})
# Criando agentes
meg = doc.agent('kagg:Meg Risdal', {model.PROV_TYPE : 'prov:Person', 'kagg:position': 'Product Manager', model.PROV_LOCATION : 'Los Angeles, California, United States',
                                    'kagg:user' : 'mrisdal', 'kagg:profile' : 'https://www.kaggle.com/mrisdal'})
# Criando atividades
#doc.activity('kagg:Scrap Text and Metadata', '2016-10-25', '2016-11-25', {'kagg:source' : '244 websites'})
doc.activity('kagg:Scrap Text and Metadata', '25/10/2016', '25/11/2016', {'kagg:source' : '244 websites'})
# TODO adicionar atividades, propriedades e relacionamentos:
# 244 websites tagged as "bullshit" by the BS Detector Chrome Extension by Daniel Sieradski.
# The data was pulled using the webhose.io API;
# Relacionamentos
zip.wasDerivedFrom(dataset)
csv.wasDerivedFrom(zip)
print(doc.get_provn())
# TODO nova seção:
# Save the provenance graph
from prov.dot import prov_to_dot
dot = prov_to_dot(doc)
dot.write_png('fake-news-dataset-prov.png')
# Visualize the graph file
from IPython.display import Image
Image('fake-news-dataset-prov.png')
# https://www.kaggle.com/mrisdal/fake-news